# SimpleCNN + Dataset class

In [8]:
# 사용자 정의 Dataset 클래스 추가
import os
from torch.utils.data import Dataset
from PIL import Image

class CustomImageDataset(Dataset):
    def __init__(self, root_dir, label_map, transform=None):
        self.samples = []
        self.transform = transform
        self.label_map = label_map  # 예: {'cat': 0, 'dog': 1}

        for class_name, label in label_map.items():
            class_path = os.path.join(root_dir, class_name)
            for fname in os.listdir(class_path):
                if fname.lower().endswith(('.jpg', '.png')):
                    self.samples.append((os.path.join(class_path, fname), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import random
import shutil

# 1. 하이퍼파라미터 및 설정
BATCH_SIZE = 4
EPOCHS = 30
LR = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE=", DEVICE)

# 2. 데이터 전처리
# 이미지를 전처리(Preprocessing) 하기 위한 연속된 변환 작업(transform pipeline) 을 정의
transform = transforms.Compose([
    transforms.Resize((128, 128)),      # 이미지를 고정 크기로 설정
    transforms.ToTensor(),              # 이미지를 PyTorch 텐서로 변환
    transforms.Normalize([0.5], [0.5])  # 빠르고 안정적인 학습을 위한 정규화(0~1 -> -1~1), (x-0.5)/0.5
])
data_path = "C:/Users/602-18/YOLO/Learning/class/before/dataset/carrot"
classes = ['GOOD', 'BAD']
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')

for cls in classes:
    class_dir = os.path.join(data_path, cls)
    if not os.path.isdir(class_dir):
        print(f"[❌] 디렉토리 없음: {class_dir}")
        continue

    images = [f for f in os.listdir(class_dir)
              if os.path.isfile(os.path.join(class_dir, f)) and f.lower().endswith(image_extensions)]

    print(f"[✅] {cls} 클래스 - 이미지 수: {len(images)}")



DEVICE= cpu
[✅] GOOD 클래스 - 이미지 수: 527
[✅] BAD 클래스 - 이미지 수: 550


In [10]:
# 클래스 목록
classes = ['GOOD', 'BAD']

# 분할 저장 경로
dest_root = 'C:/Users/602-18/YOLO/Learning/class/before/dataset_split/carrot'

# # 4. 분할 비율
# split_ratio = [0.8, 0.19, 0.01]
# splits = ['train', 'val', 'test']

# # 5. 대상 폴더 구조 생성: .../carrot/train/GOOD 등
# for split in splits:
#     for cls in classes:
#         split_cls_dir = os.path.join(dest_root, split, cls)
#         os.makedirs(split_cls_dir, exist_ok=True)

# # 6. 이미지 분할 및 복사
# for cls in classes:
#     src_dir = os.path.join(data_path, cls)
#     if not os.path.isdir(src_dir):
#         print(f"[❌] 디렉토리 없음: {src_dir}")
#         continue

#     images = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))]
#     random.shuffle(images)

#     total = len(images)
#     train_end = int(split_ratio[0] * total)
#     val_end = train_end + int(split_ratio[1] * total)

#     split_files = {
#         'train': images[:train_end],
#         'val': images[train_end:val_end],
#         'test': images[val_end:]
#     }

#     for split, file_list in split_files.items():
#         for img in file_list:
#             src = os.path.join(src_dir, img)
#             dst = os.path.join(dest_root, split, cls, img)
#             os.makedirs(os.path.dirname(dst), exist_ok=True)
#             shutil.copy(src, dst)

# print("✅ 이미지 분할 및 복사가 완료되었습니다.")

In [ ]:
import torchvision.transforms.functional as F
val_dir = 'C:/Users/602-18/YOLO/Learning/class/before/dataset_split/carrot/val'

num_aug_per_image = 2
def augment_image(image):
    if random.random() > 0.5:
        image = F.hflip(image)                           # 반전
    if random.random() > 0.5:
        angle = random.uniform(-30, 30)
        image = F.rotate(image, angle)                   # 회전
    if random.random() > 0.5:
        brightness = random.uniform(0.7, 1.3)
        contrast = random.uniform(0.7, 1.3)
        image = F.adjust_brightness(image, brightness)   # 밝기 조절
        image = F.adjust_contrast(image, contrast)       # 명암 조절
    return image

# 이미지 확장자
image_exts = ('.jpg', '.jpeg', '.png')

for cls in classes:
    class_dir = os.path.join(val_dir, cls)
    images = [f for f in os.listdir(class_dir) if f.lower().endswith(image_exts)]

    print(f"[📁] 클래스: {cls} | 이미지 수: {len(images)}")

    for img_name in images:
        img_path = os.path.join(class_dir, img_name)

        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"[⚠️] 이미지 열기 실패: {img_path}, 에러: {e}")
            continue

        for i in range(num_aug_per_image):
            aug_img = augment_image(image)
            base, ext = os.path.splitext(img_name)
            new_filename = f"aug_{base}_{i}{ext}"
            new_path = os.path.join(class_dir, new_filename)
            aug_img.save(new_path)

[📁] 클래스: GOOD | 이미지 수: 100
[📁] 클래스: BAD | 이미지 수: 104
✅ val 이미지에 대한 증강 완료 및 저장됨.


In [ ]:
def add_noise(img, stddev=50):
    arr = np.array(img).astype(np.float32)
    noise = np.random.normal(0, stddev, arr.shape)
    noisy_arr = np.clip(arr + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy_arr)

def apply_affine(img):
    width, height = img.size
    coeffs = (1, 0.2, -10,   # a, b, c
              0.1, 1, -5)    # d, e, f
    return img.transform((width, height), Image.AFFINE, coeffs, resample=Image.BICUBIC)

def apply_rotation(img, angle=4):
    return img.rotate(angle, resample=Image.BICUBIC, expand=True).crop((0, 0, img.size[0], img.size[1]))

def apply_random_crop_resize(img, crop_ratio=0.9):
    w, h = img.size
    crop_w, crop_h = int(w * crop_ratio), int(h * crop_ratio)
    left = np.random.randint(0, w - crop_w + 1)
    top = np.random.randint(0, h - crop_h + 1)
    cropped = img.crop((left, top, left + crop_w, top + crop_h))
    return cropped

In [12]:
# label_map 정의
label_map = {'BAD': 0, 'GOOD': 1}
class_names = list(label_map.keys())

# 커스텀 Dataset 적용
train_dataset = CustomImageDataset(root_dir=dest_root+'/train', label_map=label_map, transform=transform)
valid_dataset = CustomImageDataset(root_dir=dest_root+'/val', label_map=label_map, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)  # 모델이 순서에 영향을 받지 않도록 매 epoch마다 무작위로 섞는다
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False) # 데이터 순서 고정



In [13]:
# 3. 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            # nn.Conv2d(3채널(RGB), 필터수, 필터크기, stride=1, padding=0)
            nn.Conv2d(3, 16, 3, padding=1),  # 128x128x3 -> 128x128x16, padding=1은 1픽셀 추가하여 출력크기 유지
            nn.ReLU(),
            nn.MaxPool2d(2),                # -> 64x64x16, 이미지 크기를 1/2로 축소(국소적 특징 요약)
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                # -> 32x32x32
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 224),  # 입력은 CNN에서 전달된 크기, 출력은 보통 64, 128, 256, 512 등
            nn.ReLU(),
            nn.Linear(224, 2)   # 최종 출력이 1이면 Sigmoid연결, 2이면 Softmax연결
            # BCEWithLogitsLoss() (또는 BCELoss + Sigmoid),	CrossEntropyLoss() (Softmax 포함)
        )

    def forward(self, x):
        return self.fc(self.conv(x))

model = SimpleCNN().to(DEVICE)
criterion = nn.CrossEntropyLoss()  # Softmax 포함
optimizer = optim.Adam(model.parameters(), lr=LR)

In [15]:
# 4. 학습 및 시각화용 리스트
import time  # 추가

train_acc_list, val_acc_list = [], []

for epoch in range(EPOCHS):
    start_time = time.time()  # ⏱️ 시작 시간 기록

    model.train()
    correct, total, loss_total = 0, 0, 0
    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        loss_total += loss.item()
        correct += (outputs.argmax(1) == y).sum().item()
        total += y.size(0)
    train_acc = correct / total
    train_acc_list.append(train_acc)

    # 검증
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = model(x)
            correct += (outputs.argmax(1) == y).sum().item()
            total += y.size(0)
    val_acc = correct / total
    val_acc_list.append(val_acc)

    elapsed_time = time.time() - start_time  # ⏱️ 경과 시간 계산

    print(f"Epoch {epoch+1} | Loss: {loss_total:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Time: {elapsed_time:.2f} sec")

Epoch 1 | Loss: 10.8846 | Train Acc: 0.9837 | Val Acc: 0.8284 | Time: 41.50 sec
Epoch 2 | Loss: 0.0040 | Train Acc: 1.0000 | Val Acc: 0.8301 | Time: 41.53 sec
Epoch 3 | Loss: 0.0008 | Train Acc: 1.0000 | Val Acc: 0.8268 | Time: 42.23 sec
Epoch 4 | Loss: 0.0004 | Train Acc: 1.0000 | Val Acc: 0.8235 | Time: 43.20 sec
Epoch 5 | Loss: 0.0002 | Train Acc: 1.0000 | Val Acc: 0.8235 | Time: 42.62 sec
Epoch 6 | Loss: 0.0001 | Train Acc: 1.0000 | Val Acc: 0.8219 | Time: 42.57 sec
Epoch 7 | Loss: 0.0001 | Train Acc: 1.0000 | Val Acc: 0.8186 | Time: 42.98 sec
Epoch 8 | Loss: 0.0001 | Train Acc: 1.0000 | Val Acc: 0.8154 | Time: 42.69 sec
Epoch 9 | Loss: 0.0000 | Train Acc: 1.0000 | Val Acc: 0.8121 | Time: 42.43 sec
Epoch 10 | Loss: 0.0000 | Train Acc: 1.0000 | Val Acc: 0.8088 | Time: 43.85 sec
Epoch 11 | Loss: 0.0000 | Train Acc: 1.0000 | Val Acc: 0.8072 | Time: 43.45 sec
Epoch 12 | Loss: 0.0000 | Train Acc: 1.0000 | Val Acc: 0.8072 | Time: 43.43 sec
Epoch 13 | Loss: 0.0000 | Train Acc: 1.0000 | Va

In [2]:
import matplotlib.pyplot as plt

# 5. 학습 시각화
plt.plot(train_acc_list, label='Train Accuracy')
plt.plot(val_acc_list, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Progress')
plt.legend()
plt.grid(True)
plt.show()

# 6. 모델 저장
torch.save(model.state_dict(), "carrot_cnn.pth")

NameError: name 'train_acc_list' is not defined